In [3]:
from qiskit_nature.second_q.operators import ElectronicIntegrals, FermionicOp ,SparseLabelOp, PolynomialTensor, tensor_ordering
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
#from qiskit_nature.second_q.circuit.library import UCCSD

########### UCC modules #######
#from qiskit_nature.second_q.circuit import QubitConverter
from qiskit_nature.second_q.circuit.library.ansatzes import UCC
from qiskit_nature.second_q.circuit.library.initial_states import HartreeFock


########### VQE Modules #######
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit.quantum_info import Statevector


########### OLD VQE Modules #######
from qiskit_algorithms import VQE
from qiskit_aer.primitives import Estimator as AerEstimator


######### Optimiser Modules #########
from qiskit_algorithms.optimizers import COBYLA , SLSQP , L_BFGS_B


from qiskit.quantum_info import SparsePauliOp

from qiskit.circuit.library import NLocal,EfficientSU2
######### PySCF + others #########

import sys
sys.argv.append("--quiet")  # For Vayesta
import numpy as np
from pyscf import gto, scf, lib, ao2mo
# from vayesta.lattmod import Hubbard1D, LatticeRHF
import itertools
import math
import array_to_latex as a2l
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
import pandas as pd 
#import matplotlib.pyplot as plt
from qiskit.quantum_info.operators import Operator
import qiskit
from pyscf import cc as cupclus

from src import classical_shadow
# from src import swap_test
# from src import observables
import cmath
import pyscf
from pyscf import gto, scf, ao2mo, fci, ci
import os.path

# import julia
# julia.install()
# from julia.api import Julia
# jl = Julia(compiled_modules=False)
# from julia import Main
# from julia import ITensors

# Main.include("../Quantum_cluster/src/julia_functions.jl")


# from src import Lan_coeffs

import ffsim
import numpy as np
from pyscf import cc

import scipy.optimize
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit_ibm_runtime.fake_provider import FakeBrisbane
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator
import pickle 

from qiskit_nature.second_q.circuit.library.ansatzes.utils import generate_fermionic_excitations 
def excitations_tuples_to_strings(p_exc,fermi_vac):
        p_exc_list =[]
        for exc_det_count in range(len(p_exc)):
            # print(p_exc[exc_det_count])
            # print(fermi_vac)
            fermi_vac_exc=fermi_vac
            for n_exc in range(len(p_exc[exc_det_count][0])):
                index1 = -1-p_exc[exc_det_count][0][n_exc]
                index2 = -1-p_exc[exc_det_count][1][n_exc]
                # print(fermi_vac_exc[index1])
                if index1==-1:
                    s = fermi_vac_exc[:index1] + "0"
                else:
                    s = fermi_vac_exc[:index1] + "0" + fermi_vac_exc[index1 + 1:]    

                if index2==-1:    
                    s2 = s[:index2] + "1"
                else:
                    s2 = s[:index2] + "1" + s[index2 + 1:]        
                # print(s)
                # print(s2)
                fermi_vac_exc = s2
            # print(fermi_vac_exc)   
            p_exc_list.append(fermi_vac_exc) 
        return p_exc_list

In [4]:
nsite=4
d=1.0

my_atom=[("H 0. 0. %f" % xyz) for xyz in [d*x for x in list(range(nsite))]]

num_e = nsite




mol = gto.M(
    #atom = 'H 0 0 0; H 0 0 1.0; H 0 0 2.0; H 0 0 3.0',  # in Angstrom
    #atom = 'H 0 0 0; H 0 0 2.',#; H 0 0 4.0; H 0 0 6.0',  # in Angstrom
    #atom = 'H 0 0 0; H 0 0 1.55; H 0 0 3.1; H 0 0 4.65',  # in Angstrom
    #atom = 'H 0 0 0; H 0 0 2.',#; H 0 0 4.0; H 0 0 6.0',  # in Angstrom
    atom=my_atom,
    basis = 'sto-3g',
    symmetry = True,
    verbose = 3
)
nelec = mol.nelec
myhf = mol.RHF().run()
assert(myhf.converged)




cc = cupclus.CCSD(myhf)
cc.kernel()
assert cc.converged
print('CCSD total energy: {}'.format(cc.e_tot))
CCSD_energy = cc.e_tot





one_body = myhf.mo_coeff.T @ myhf.get_hcore() @ myhf.mo_coeff
eri = ao2mo.kernel(myhf._eri, (myhf.mo_coeff, myhf.mo_coeff,myhf.mo_coeff,myhf.mo_coeff), compact=False)
two_body = eri.reshape((myhf.mo_coeff.shape[-1],) * 4)

# Constructing the electronic hamiltonian in second quantised representation
integrals = ElectronicIntegrals.from_raw_integrals(h1_a=one_body, h1_b = one_body, h2_aa=two_body, h2_bb= two_body, h2_ba= two_body , auto_index_order=True) 

# Defining the many body electronic hamiltionian in second quantised representation

h_elec = ElectronicEnergy(integrals, constants = {'nuclear_repulsion_energy':mol.energy_nuc()}).second_q_op()       
mapper = JordanWignerMapper()
qubit_ham = mapper.map(h_elec) # this performs the JW transformation to qubit representation

my_fci = pyscf.fci.FCI(myhf)
my_fci.kernel()
print('FCI total energy: {}'.format(my_fci.e_tot))

converged SCF energy = -2.09854593699772


/home/connorlenihan/.local/lib/python3.8/site-packages/pyscf/solvent/ddpcm.py:38: UserWarning: Module ddPCM is under testing
  warnings.warn('Module ddPCM is under testing')
SMD module is not available. You can compile this module with cmake option "-DENABLE_SMD=ON"<class 'pyscf.cc.ccsd.CCSD'> does not have attributes  converged


E(CCSD) = -2.166379521639531  E_corr = -0.0678335846418133
CCSD total energy: -2.1663795216395307
FCI total energy: -2.166387448632114


In [ ]:


mol_data = ffsim.MolecularData.from_scf(myhf)
norb = mol_data.norb
nelec = mol_data.nelec
mol_hamiltonian = mol_data.hamiltonian

# norb = myhf.mo_coeff.shape[-1]

# Get CCSD t2 amplitudes for initializing the ansatz
ccsd = cc.CCSD(
    myhf
).run()

# Construct UCJ operator
n_reps = 1
operator = ffsim.UCJOpSpinBalanced.from_t_amplitudes(ccsd.t2, n_reps=n_reps)

# Construct the Hartree-Fock state to use as the reference state
reference_state = ffsim.hartree_fock_state(norb, nelec)

# Apply the operator to the reference state
ansatz_state = ffsim.apply_unitary(reference_state, operator, norb=norb, nelec=nelec)

# Compute the energy ⟨ψ|H|ψ⟩ of the ansatz state
hamiltonian = ffsim.linear_operator(mol_hamiltonian, norb=norb, nelec=nelec)
energy = np.real(np.vdot(ansatz_state, hamiltonian @ ansatz_state))
print(f"Energy at initialization: {energy}")



import scipy.optimize


def fun(x):
    # Initialize the ansatz operator from the parameter vector
    operator = ffsim.UCJOpSpinBalanced.from_parameters(x, norb=norb, n_reps=n_reps)
    # Apply the ansatz operator to the reference state
    final_state = ffsim.apply_unitary(reference_state, operator, norb=norb, nelec=nelec)
    # Return the energy ⟨ψ|H|ψ⟩ of the ansatz state
    return np.real(np.vdot(final_state, hamiltonian @ final_state))


result = scipy.optimize.minimize(
    fun, x0=operator.to_parameters(), method="L-BFGS-B", options=dict(maxiter=10)
)

print(f"Number of parameters: {len(result.x)}")
print(result)

pairs_aa = [(p, p + 1) for p in range(norb - 1)]
pairs_ab = [(p, p) for p in range(norb)]
interaction_pairs = (pairs_aa, pairs_ab)


def fun(x):
    operator = ffsim.UCJOpSpinBalanced.from_parameters(
        x, norb=norb, n_reps=n_reps, interaction_pairs=interaction_pairs
    )
    final_state = ffsim.apply_unitary(reference_state, operator, norb=norb, nelec=nelec)
    # print(operator.)
    return np.real(np.vdot(final_state, hamiltonian @ final_state))

result = scipy.optimize.minimize(
    fun,
    x0=operator.to_parameters(interaction_pairs=interaction_pairs),
    method="L-BFGS-B",
    options=dict(maxiter=30),
)
print(f"Number of parameters: {len(result.x)}")
print(result)

NameError: name 'CCSD' is not defined

In [35]:
from collections import defaultdict

from ffsim.optimize import minimize_linear_method


mol_data = ffsim.MolecularData.from_scf(myhf)
norb = mol_data.norb
nelec = mol_data.nelec
mol_hamiltonian = mol_data.hamiltonian
hamiltonian = ffsim.linear_operator(mol_hamiltonian, norb=norb, nelec=nelec)
# print(norb,nelec)

n_reps=1


pairs_aa = [(p, p + 1) for p in range(norb - 1)]
pairs_ab = [(p, p) for p in range(norb)]
interaction_pairs = (pairs_aa, pairs_ab)

ccsd = cupclus.CCSD(
    myhf
).run()

operator = ffsim.UCJOpSpinBalanced.from_t_amplitudes(ccsd.t2, n_reps=n_reps)

# Construct the Hartree-Fock state to use as the reference state
reference_state = ffsim.hartree_fock_state(norb, nelec)

# Define function that converts a list of parameters to the corresponding state vector
def params_to_vec(x: np.ndarray) -> np.ndarray:
    operator = ffsim.UCJOpSpinBalanced.from_parameters(
        x, norb=norb, n_reps=n_reps, interaction_pairs=interaction_pairs
    )
    return ffsim.apply_unitary(reference_state, operator, norb=norb, nelec=nelec)


# Define a callback function used to save optimization information (this is optional)
info = defaultdict(list)


def callback(intermediate_result: scipy.optimize.OptimizeResult):
    # The callback function is called after each iteration. It accepts
    # an OptimizeResult object storing the parameters and function value at
    # the current iteration, and possibly other information
    info["x"].append(intermediate_result.x)
    info["fun"].append(intermediate_result.fun)
    if hasattr(intermediate_result, "jac"):
        info["jac"].append(intermediate_result.jac)
    if hasattr(intermediate_result, "regularization"):
        info["regularization"].append(intermediate_result.regularization)
    if hasattr(intermediate_result, "variation"):
        info["variation"].append(intermediate_result.variation)


# Optimize with the linear method
result = minimize_linear_method(
    params_to_vec,
    hamiltonian,
    x0=operator.to_parameters(interaction_pairs=interaction_pairs),
    maxiter=1000,
    callback=callback,
)

# Print some information
print(f"Number of parameters: {len(result.x)}")
print(result)
print()
for i, (fun, jac, regularization, variation) in enumerate(
    zip(info["fun"], info["jac"], info["regularization"], info["variation"])
):
    print(f"Iteration {i + 1}")
    print(f"    Energy: {fun}")
    print(f"    Norm of gradient: {np.linalg.norm(jac)}")
    print(f"    Regularization hyperparameter: {np.linalg.norm(regularization)}")
    print(f"    Variation hyperparameter: {np.linalg.norm(variation)}")

E(CCSD) = -3.235677082524778  E_corr = -0.1001448685584544
Number of parameters: 47
 message: Convergence: Norm of projected gradient <= gtol.
 success: True
     fun: -3.223462624217581
       x: [ 9.058e-01  1.260e-02 ... -4.342e-01 -4.751e-01]
     nit: 45
     jac: [ 2.475e-06  1.515e-06 ... -2.585e-06  1.721e-06]
    nfev: 4887
    njev: 46
  nlinop: 2725

Iteration 1
    Energy: -3.166137380413304
    Norm of gradient: 0.020454936962354967
    Regularization hyperparameter: 0.009542536439994097
    Variation hyperparameter: 0.9996207411501564
Iteration 2
    Energy: -3.1680656471986377
    Norm of gradient: 0.018243576511182965
    Regularization hyperparameter: 0.025621771165055082
    Variation hyperparameter: 0.9996207890755344
Iteration 3
    Energy: -3.1715795741241637
    Norm of gradient: 0.036370449240911325
    Regularization hyperparameter: 0.0050261583929123365
    Variation hyperparameter: 0.9996207712229116
Iteration 4
    Energy: -3.1789524888434255
    Norm of grad

In [39]:
print(result.x)
optimal_params_1_6 = [0.90578388,0.01260129,1.10511399,-0.42179577,0.38170124,0.30454347,-0.41674204,1.64278538,0.57473984
,-0.1544528,-0.5014813,0.40658773,0.40095168,-0.22142656,-0.18737288,0.10149377,0.70119982,-0.9306893
,0.84867086,0.09807601,-0.90995703,-1.31882741,0.20747538,-0.29524065,0.83492754,-0.22271534,1.67519111
,0.01454644,-0.40021452,1.32279179,-0.37140237,-0.56270401,-0.07039741,0.37229892,-0.5294931,0.32664418
,0.20105547,-0.11040596,-0.22252456,-0.13636446,0.15419335,-0.42722945,-0.53570156,0.00931705,-0.07141593
,-0.43424842,-0.47512572]
with open('LUCJ_nn_params_n_reps_'+str(n_reps)+'_Hydrogen_'+str(nsite)+'_atom.pkl', 'wb') as f:
    pickle.dump(optimal_params_1_6, f)


[ 0.90578388  0.01260129  1.10511399 -0.42179577  0.38170124  0.30454347 -0.41674204  1.64278538  0.57473984
 -0.1544528  -0.5014813   0.40658773  0.40095168 -0.22142656 -0.18737288  0.10149377  0.70119982 -0.9306893
  0.84867086  0.09807601 -0.90995703 -1.31882741  0.20747538 -0.29524065  0.83492754 -0.22271534  1.67519111
  0.01454644 -0.40021452  1.32279179 -0.37140237 -0.56270401 -0.07039741  0.37229892 -0.5294931   0.32664418
  0.20105547 -0.11040596 -0.22252456 -0.13636446  0.15419335 -0.42722945 -0.53570156  0.00931705 -0.07141593
 -0.43424842 -0.47512572]


In [ ]:
# print(result.x)
optimal_params_1_2=[-1.96964782,-1.20855392,-0.81585484,0.42315576,0.08903292,-0.35243734,-0.35243734]
with open('LUCJ_nn_params_n_reps_'+str(n_reps)+'_Hydrogen_'+str(nsite)+'_atom.pkl', 'wb') as f:
    pickle.dump(optimal_params_1_2, f)


In [ ]:
# print(result.x)
optimal_params_1 = [5.20238071e-01,-1.46440164e+00,6.32596863e-02,2.91782243e-01,-6.90402316e-02,8.06905575e-01,9.24387942e-01
,-4.91323970e-01,-6.34107959e-01,-1.38679445e+00,8.44947515e-01,4.94526211e-03,9.42290059e-01,-5.37566280e-01
,-5.66186135e-01,3.39029312e-01,3.24740432e-01,-1.55763138e-01,-3.05035972e-01,-6.58872439e-01,-5.08110967e-01
,-5.57867419e-04,-4.23376768e-01]


[ 5.20238071e-01 -1.46440164e+00  6.32596863e-02  2.91782243e-01 -6.90402316e-02  8.06905575e-01  9.24387942e-01
 -4.91323970e-01 -6.34107959e-01 -1.38679445e+00  8.44947515e-01  4.94526211e-03  9.42290059e-01 -5.37566280e-01
 -5.66186135e-01  3.39029312e-01  3.24740432e-01 -1.55763138e-01 -3.05035972e-01 -6.58872439e-01 -5.08110967e-01
 -5.57867419e-04 -4.23376768e-01]


In [13]:
mol_data = ffsim.MolecularData.from_scf(myhf)
norb = mol_data.norb
nelec = mol_data.nelec
nocc = nelec[0]
nvir=norb-nocc
mol_hamiltonian = mol_data.hamiltonian

# Construct UCJ operator
n_reps = 1

optimal_params_1 = np.array([5.20238071e-01,-1.46440164e+00,6.32596863e-02,2.91782243e-01,-6.90402316e-02,8.06905575e-01,9.24387942e-01,-4.91323970e-01,-6.34107959e-01,-1.38679445e+00,8.44947515e-01,4.94526211e-03,9.42290059e-01,-5.37566280e-01,-5.66186135e-01,3.39029312e-01,3.24740432e-01,-1.55763138e-01,-3.05035972e-01,-6.58872439e-01,-5.08110967e-01,-5.57867419e-04,-4.23376768e-01])


with open('LUCJ_nn_params_n_reps_1_Hydrogen_4_atom.pkl', 'wb') as f:
    pickle.dump(optimal_params_1, f)

pairs_aa = [(p, p + 1) for p in range(norb - 1)]
pairs_ab = [(p, p) for p in range(norb)]
interaction_pairs = (pairs_aa, pairs_ab)
operator = ffsim.UCJOpSpinBalanced.from_parameters(
        optimal_params_1, norb=norb, n_reps=n_reps, interaction_pairs=interaction_pairs
    )

reference_state = ffsim.hartree_fock_state(norb, nelec)

# Apply the operator to the reference state
ansatz_state = ffsim.apply_unitary(reference_state, operator, norb=norb, nelec=nelec)

# Compute the energy ⟨ψ|H|ψ⟩ of the ansatz state
hamiltonian = ffsim.linear_operator(mol_hamiltonian, norb=norb, nelec=nelec)
energy = np.real(np.vdot(ansatz_state, hamiltonian @ ansatz_state))
print(f"Energy: {energy}")



print(nelec)
# qubits = 2*norb
qubits = qiskit.QuantumRegister(2 * norb, name="q")
circuit = qiskit.QuantumCircuit(qubits)
circuit.append(ffsim.qiskit.PrepareHartreeFockJW(norb, nelec),qubits)
circuit.append(ffsim.qiskit.UCJOpSpinBalancedJW(operator),qubits)

# get backend
# provider = IBMProvider()
backend = FakeBrisbane()

qc_compiled = transpile(circuit, backend,optimization_level=3)

print("depth:",qc_compiled.depth(),"\ntotal gates: ",qc_compiled.size(),"\ngate breakdown: ",qc_compiled.count_ops())


Energy: -2.161876167816394
(2, 2)
depth: 178 
total gates:  716 
gate breakdown:  OrderedDict([('rz', 331), ('sx', 270), ('ecr', 104), ('x', 11)])


In [7]:
with open('cliff_list_actual.pkl', 'rb') as f:
    clifford_list_2 = pickle.load(f)

In [14]:
def cbssp_unitary(stateA,stateB,imag=False):
    '''
    create superposition (sp) of computaional basis states (cbs), given by strings of 1s and 0s
    returns qiskit circuit creating the sp
    '''

    both1=[]
    a1=[]
    b1=[]
    if len(stateA) != len(stateB):
        sys.exit("states diff lengths")
    for n in range(len(stateA)):
        if stateA[n] =="1" and stateB[n]=="1":
            both1.append(len(stateA)-1-n)
        if stateA[n] =="1" and stateB[n]=="0":
            a1.append(len(stateA)-1-n)    
        if stateA[n] =="0" and stateB[n]=="1":
            b1.append(len(stateA)-1-n)    
    # print(both1)        
    # print(a1)
    # print(b1)
    sp_circ=qiskit.QuantumCircuit(len(stateA))
    for n in both1:
        sp_circ.x(n)
    if len(a1)!=0:
        sp_circ.h(a1[0])
        if imag:
            sp_circ.s(a1[0])
        for n in np.arange(1,len(a1[:])):
            #print(n,a1[0],a1[n])
            sp_circ.cx(a1[0],a1[n])

        sp_circ.x(a1[0])
        for n in np.arange(0,len(b1[:])):
            #print(n)
            sp_circ.cx(a1[0],b1[n])    
        sp_circ.x(a1[0])
    elif len(a1)==0 and len(b1)!=0:
        sp_circ.h(b1[0])
        if imag:
            sp_circ.s(b1[0])
        for n in np.arange(1,len(b1[:])):
            sp_circ.cx(b1[0],b1[n])
    return sp_circ,a1[0]

backend = FakeBrisbane()
# backend = AerSimulator()

operator = ffsim.UCJOpSpinBalanced.from_parameters(
        optimal_params_1, norb=norb, n_reps=n_reps, interaction_pairs=interaction_pairs
    )
fermi_vac='00110011'
nshots = 8000

with open('cliff_list_actual.pkl', 'rb') as f:
    clifford_list_2 = pickle.load(f)
meaurement_result = []
for cliff_count in range(nshots):
    rand_cliff = clifford_list_2[cliff_count]
    circuit = qiskit.QuantumCircuit(qubits)
    # circuit.append(ffsim.qiskit.PrepareHartreeFockJW(norb, nelec),qubits)
    sp_circ,a1 = cbssp_unitary('00110011',"0"*8)
    circuit.append(sp_circ,qubits)
    circuit.append(ffsim.qiskit.UCJOpSpinBalancedJW(operator),qubits)
    circuit.append(qiskit.quantum_info.Clifford.from_dict(rand_cliff).to_instruction(),qubits)
    circuit.measure_all()
    qiskit.qasm2.dump(circuit, "qasm_circuits/classical_shadow_circuit_"+str(cliff_count)+".qasm")
    circuit2=qiskit.QuantumCircuit.from_qasm_file("qasm_circuits/classical_shadow_circuit_"+str(cliff_count)+".qasm")
    qc_compiled = transpile(circuit2, backend)

    # # Execute the circuit on the qasm simulator.
    # # We've set the number of repeats of the circuit
    # # to be 1024, which is the default.
    # job_sim = backend.run(qc_compiled, shots=1)

    # # Grab the results from the job.
    # result_sim = job_sim.result()
    # meaurement_result.append(list(result_sim.get_counts().keys())[0])
    # # print(result_sim.get_counts())



In [ ]:
import qiskit
from qiskit import transpile
from qiskit_ibm_runtime.fake_provider import FakeBrisbane
import pickle

nshots = 8000
backend = FakeBrisbane()
measurement_result = []
for cliff_count in range(nshots):
    circuit2=qiskit.QuantumCircuit.from_qasm_file("qasm_circuits/classical_shadow_circuit_"+str(cliff_count)+".qasm")
    qc_compiled = transpile(circuit2, backend)
    job_sim = backend.run(qc_compiled, shots=1)
    result_sim = job_sim.result()
    measurement_result.append(list(result_sim.get_counts().keys())[0])

with open('measurement_results_shadows.pkl', 'wb') as f:
    pickle.dump(measurement_result, f)

f_name="measurement_results_shadows.txt"
f = open(f_name, "a")
for m in measurement_result:
    f.write(str(m)+"\n")
f.close()   



In [6]:
print(measurement_result)

['11101000', '11111010', '01101111', '00100011', '11010000']


In [7]:
with open('measurement_results.pkl', 'rb') as f:
    measurement_result = pickle.load(f)
print(measurement_result)

['11101000', '11111010', '01101111', '00100011', '11010000']


In [9]:
from pyscf.ci import cisd

print(meaurement_result)
cliffords_for_Us = [qiskit.quantum_info.Clifford.from_dict(rand_cliff) for rand_cliff in clifford_list_2[:nshots]]
shadow = classical_shadow.Classical_shadow("Clifford",8,meaurement_result,cliffords_for_Us)

t1addr, t1sign = cisd.tn_addrs_signs(norb, nocc, 1)

op_list=[]
index_a=0#"0"*SV_basis_0.num_qubits#int(list(SV_basis_0.to_dict().keys())[0],2)
###index for HF state
index_b=int(fermi_vac,2)
op_list.append([1.,index_b,index_a])
result,error = shadow.evaluate_operator_overlap_stabilizer_perp(op_list,samples_per_estimator=int(nshots//3),num_estimators = 3)
c0_cs=result[0]


ab_exc = generate_fermionic_excitations(num_excitations=2,max_spin_excitation=1, num_spatial_orbitals=norb, num_particles=nelec, alpha_spin=True, beta_spin=True, generalized=False, preserve_spin=True)
ab_exc_strings = excitations_tuples_to_strings(ab_exc,fermi_vac)
### construct the labels for operators of the form |index_a><index_b| + |index_b><index_a| (for the real part) + imaginary part
ab_int_norm_results=[]
print("ab_exc_strings: ",ab_exc_strings)
op_list=[]
for ab in ab_exc_strings:
    index_a=0#"0"*SV_basis_0.num_qubits#int(list(SV_basis_0.to_dict().keys())[0],2)
    index_b=int(ab,2)
    # print(index_b)
    op_list.append([1.,index_b,index_a])
result,error = shadow.evaluate_operator_overlap_stabilizer_perp(op_list,samples_per_estimator=int(nshots//3),num_estimators = 3)
c2_cs_ab = np.einsum("i,j,ij->ij", t1sign, t1sign, np.reshape(result,(len(t1addr),len(t1addr))))
c2_cs_ab = np.reshape(c2_cs_ab,(nocc, nvir, nocc, nvir)).transpose(0, 2, 1, 3)

c2_cs_rs = c2_cs_ab/c0_cs
g_ovvo = two_body[:nocc,nocc:,nocc:,:nocc]
e_singles=0.
e_doubles = 2 * np.einsum("ijab,iabj->", c2_cs_rs, g_ovvo) - np.einsum("ijab,ibaj->", c2_cs_rs, g_ovvo)
e_proj = myhf.e_tot + e_singles + e_doubles
print(e_proj)

['10001100', '01001111', '01000011', '10001010', '10100101', '11011110', '01111100', '00111001', '10100000', '10001011', '01001001', '01100110', '11100100', '10001010', '10000011', '01011101', '10000010', '10010010', '00111101', '10111000', '10110000', '11010010', '10111111', '10000011', '01100010', '11110100', '10001100', '11000001', '11000010', '00010111', '00001001', '01101110', '00010101', '00001101', '10010101', '11101111', '00101001', '01001010', '00100100', '11011000', '11110011', '00111110', '00110011', '00101011', '01101001', '10111111', '11111101', '11100001', '00100100', '11001000', '11101100', '11101111', '11010110', '11011011', '00110001', '00101111', '01001001', '10010100', '01100000', '00101101', '11110100', '10110110', '11001101', '00110111', '01101111', '01110010', '11000111', '00000001', '01001000', '00010011', '00111000', '11011111', '00010110', '00110110', '10001100', '00000001', '01001011', '00101011', '00000001', '10011111', '11000110', '00101010', '00110101', '01

/home/connorlenihan/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:43: FutureWarning: The input object of type 'Clifford' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Clifford', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  result = getattr(asarray(obj), method)(*args, **kwds)
/home/connorlenihan/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **

shadow_measurements.shape:  (3, 333)
shadow_Us.shape:  (3, 333)
ab_exc_strings:  ['01100110', '10100110', '01010110', '10010110', '01101010', '10101010', '01011010', '10011010', '01100101', '10100101', '01010101', '10010101', '01101001', '10101001', '01011001', '10011001']
shadow_measurements.shape:  (3, 333)
shadow_Us.shape:  (3, 333)
(-2.1179601371529793+0.4233673138684867j)


In [42]:
my_fci.ci[0][0]

0.9500490298220006

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
import ffsim
import numpy as np
from pyscf import cc
from pyscf import gto, scf, ao2mo, fci, ci
from pyscf import cc as cupclus

import scipy.optimize

import qiskit

from qiskit import transpile
from qiskit_ibm_runtime.fake_provider import FakeBrisbane
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# from qiskit_aer import AerSimulator

from qiskit_nature.second_q.operators import ElectronicIntegrals
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy

import pickle 

backend = FakeBrisbane()
# backend = AerSimulator()


nsite=4
d=1.0

my_atom=[("H 0. 0. %f" % xyz) for xyz in [d*x for x in list(range(nsite))]]

num_e = nsite




mol = gto.M(
    #atom = 'H 0 0 0; H 0 0 1.0; H 0 0 2.0; H 0 0 3.0',  # in Angstrom
    #atom = 'H 0 0 0; H 0 0 2.',#; H 0 0 4.0; H 0 0 6.0',  # in Angstrom
    #atom = 'H 0 0 0; H 0 0 1.55; H 0 0 3.1; H 0 0 4.65',  # in Angstrom
    #atom = 'H 0 0 0; H 0 0 2.',#; H 0 0 4.0; H 0 0 6.0',  # in Angstrom
    atom=my_atom,
    basis = 'sto-3g',
    symmetry = True,
    verbose = 3
)
nelec = mol.nelec
myhf = mol.RHF().run()
assert(myhf.converged)




cc = cupclus.CCSD(myhf)
cc.kernel()
assert cc.converged
print('CCSD total energy: {}'.format(cc.e_tot))
CCSD_energy = cc.e_tot





one_body = myhf.mo_coeff.T @ myhf.get_hcore() @ myhf.mo_coeff
eri = ao2mo.kernel(myhf._eri, (myhf.mo_coeff, myhf.mo_coeff,myhf.mo_coeff,myhf.mo_coeff), compact=False)
two_body = eri.reshape((myhf.mo_coeff.shape[-1],) * 4)

# Constructing the electronic hamiltonian in second quantised representation
integrals = ElectronicIntegrals.from_raw_integrals(h1_a=one_body, h1_b = one_body, h2_aa=two_body, h2_bb= two_body, h2_ba= two_body , auto_index_order=True) 

# Defining the many body electronic hamiltionian in second quantised representation

h_elec = ElectronicEnergy(integrals, constants = {'nuclear_repulsion_energy':mol.energy_nuc()}).second_q_op()       
mapper = JordanWignerMapper()
qubit_ham = mapper.map(h_elec) # this performs the JW transformation to qubit representation


mol_data = ffsim.MolecularData.from_scf(myhf)
norb = mol_data.norb
nelec = mol_data.nelec
nocc = nelec[0]
nvir=norb-nocc


n_reps = 1

optimal_params_1 = np.array([5.20238071e-01,-1.46440164e+00,6.32596863e-02,2.91782243e-01,-6.90402316e-02,8.06905575e-01,9.24387942e-01,-4.91323970e-01,-6.34107959e-01,-1.38679445e+00,8.44947515e-01,4.94526211e-03,9.42290059e-01,-5.37566280e-01,-5.66186135e-01,3.39029312e-01,3.24740432e-01,-1.55763138e-01,-3.05035972e-01,-6.58872439e-01,-5.08110967e-01,-5.57867419e-04,-4.23376768e-01])

pairs_aa = [(p, p + 1) for p in range(norb - 1)]
pairs_ab = [(p, p) for p in range(norb)]
interaction_pairs = (pairs_aa, pairs_ab)

operator = ffsim.UCJOpSpinBalanced.from_parameters(
        optimal_params_1, norb=norb, n_reps=n_reps, interaction_pairs=interaction_pairs
    )
qubits = qiskit.QuantumRegister(2 * norb, name="q")
circuit = qiskit.QuantumCircuit(qubits)
circuit.append(ffsim.qiskit.PrepareHartreeFockJW(norb, nelec),qubits)
circuit.append(ffsim.qiskit.UCJOpSpinBalancedJW(operator),qubits)

pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
isa_psi = pm.run(circuit)
isa_observables = qubit_ham.apply_layout(isa_psi.layout)

Nshots=1000
# pauli_groups=qubit_ham.group_commuting()
estimator_options = dict(default_shots=Nshots)
estimator = Estimator(mode=backend,options=estimator_options)
job = estimator.run([(isa_psi, isa_observables)])
pub_result = job.result()[0]
# print(f"Expectation values: {pub_result.data.evs}")
print(f"Expectation values: {pub_result.data.evs + mol.energy_nuc()}")

converged SCF energy = -2.09854593699772


/home/connorlenihan/.local/lib/python3.8/site-packages/pyscf/solvent/ddpcm.py:38: UserWarning: Module ddPCM is under testing
  warnings.warn('Module ddPCM is under testing')
SMD module is not available. You can compile this module with cmake option "-DENABLE_SMD=ON"<class 'pyscf.cc.ccsd.CCSD'> does not have attributes  converged


E(CCSD) = -2.166379521639532  E_corr = -0.06783358464181331
CCSD total energy: -2.1663795216395316
Expectation values: -1.4594609391148952


In [12]:
print(f"Expectation values: {pub_result.data.evs + mol.energy_nuc()}")
print(mol.energy_nuc())

Expectation values: -1.495947385811832
2.29310124732


In [ ]:
# print(result.x)
optimal_params_2=np.array([-0.80305818,-1.87865422,-1.32542264,-0.47147446,-0.3119485,0.2329516,-0.21698665,-0.83043444,0.64905002
,-0.97117015,1.13807155,1.2368086,0.18733152,-0.75460156,0.02020656,1.40430595,0.29707656,-0.39484438
,0.44193955,-0.50953689,-0.13688567,-0.19794722,-0.85955465,-0.43298446,-1.38601635,-0.72423078,-0.20435039
,-0.53222914,0.30205987,0.63932179,-0.33603521,0.34760393,0.64758191,-2.1032701,-1.77373997,-1.12211388
,0.95467909,0.21801932,-1.06153112,0.19691338,0.8359999,0.56905493,0.93660077,0.23292747,-0.22789405
,0.64882766])

pairs_aa = [(p, p + 1) for p in range(norb - 1)]
pairs_ab = [(p, p) for p in range(norb)]
interaction_pairs = (pairs_aa, pairs_ab)

[-0.80305818 -1.87865422 -1.32542264 -0.47147446 -0.3119485   0.2329516  -0.21698665 -0.83043444  0.64905002
 -0.97117015  1.13807155  1.2368086   0.18733152 -0.75460156  0.02020656  1.40430595  0.29707656 -0.39484438
  0.44193955 -0.50953689 -0.13688567 -0.19794722 -0.85955465 -0.43298446 -1.38601635 -0.72423078 -0.20435039
 -0.53222914  0.30205987  0.63932179 -0.33603521  0.34760393  0.64758191 -2.1032701  -1.77373997 -1.12211388
  0.95467909  0.21801932 -1.06153112  0.19691338  0.8359999   0.56905493  0.93660077  0.23292747 -0.22789405
  0.64882766]


In [22]:
mol_data = ffsim.MolecularData.from_scf(myhf)
norb = mol_data.norb
nelec = mol_data.nelec
mol_hamiltonian = mol_data.hamiltonian

# Construct UCJ operator
n_reps = 2

optimal_params_2=np.array([-0.80305818,-1.87865422,-1.32542264,-0.47147446,-0.3119485,0.2329516,-0.21698665,-0.83043444,0.64905002
,-0.97117015,1.13807155,1.2368086,0.18733152,-0.75460156,0.02020656,1.40430595,0.29707656,-0.39484438
,0.44193955,-0.50953689,-0.13688567,-0.19794722,-0.85955465,-0.43298446,-1.38601635,-0.72423078,-0.20435039
,-0.53222914,0.30205987,0.63932179,-0.33603521,0.34760393,0.64758191,-2.1032701,-1.77373997,-1.12211388
,0.95467909,0.21801932,-1.06153112,0.19691338,0.8359999,0.56905493,0.93660077,0.23292747,-0.22789405
,0.64882766])

pairs_aa = [(p, p + 1) for p in range(norb - 1)]
pairs_ab = [(p, p) for p in range(norb)]
interaction_pairs = (pairs_aa, pairs_ab)
operator = ffsim.UCJOpSpinBalanced.from_parameters(
        optimal_params_2, norb=norb, n_reps=n_reps, interaction_pairs=interaction_pairs
    )

reference_state = ffsim.hartree_fock_state(norb, nelec)

# Apply the operator to the reference state
ansatz_state = ffsim.apply_unitary(reference_state, operator, norb=norb, nelec=nelec)

# Compute the energy ⟨ψ|H|ψ⟩ of the ansatz state
hamiltonian = ffsim.linear_operator(mol_hamiltonian, norb=norb, nelec=nelec)
energy = np.real(np.vdot(ansatz_state, hamiltonian @ ansatz_state))
print(f"Energy: {energy}")


print(nelec)
# qubits = 2*norb
qubits = qiskit.QuantumRegister(2 * norb, name="q")
circuit = qiskit.QuantumCircuit(qubits)
circuit.append(ffsim.qiskit.PrepareHartreeFockJW(norb, nelec),qubits)
circuit.append(ffsim.qiskit.UCJOpSpinBalancedJW(operator),qubits)

# get backend
# provider = IBMProvider()
backend = FakeBrisbane()

qc_compiled = transpile(circuit, backend,optimization_level=3)

print("depth:",qc_compiled.depth(),"\ntotal gates: ",qc_compiled.size(),"\ngate breakdown: ",qc_compiled.count_ops())

Energy: -2.166385593918806
(2, 2)
depth: 360 
total gates:  1414 
gate breakdown:  OrderedDict([('rz', 653), ('sx', 534), ('ecr', 208), ('x', 19)])


In [2]:
rand_clifford=qiskit.quantum_info.random_clifford(2*norb)
print(rand_clifford)
print(rand_clifford.to_dict())
print(qiskit.quantum_info.Clifford.from_dict(rand_clifford.to_dict()))
clifford_list=[]
for n in range(32000):
    rand_clifford=qiskit.quantum_info.random_clifford(2*norb)
    clifford_list.append(rand_clifford.to_dict())


with open('cliff_list_actual.pkl', 'wb') as f:
    pickle.dump(clifford_list, f)

Clifford: Stabilizer = ['-ZYYYIIYX', '-IZIYXXZX', '+ZYXYIYZI', '-IIXIIYIZ', '-ZIIYXYZZ', '-IXXIYYYI', '-IYYYXYXY', '+YXXYXZYY'], Destabilizer = ['+YZIXZYYI', '-XZYIYYYX', '-YZIYXYYI', '-XIZYZYII', '+IXZXXIYY', '-XIYYXIZX', '-XZZZIZXZ', '-XXIYZYZZ']
{'stabilizer': ['-ZYYYIIYX', '-IZIYXXZX', '+ZYXYIYZI', '-IIXIIYIZ', '-ZIIYXYZZ', '-IXXIYYYI', '-IYYYXYXY', '+YXXYXZYY'], 'destabilizer': ['+YZIXZYYI', '-XZYIYYYX', '-YZIYXYYI', '-XIZYZYII', '+IXZXXIYY', '-XIYYXIZX', '-XZZZIZXZ', '-XXIYZYZZ']}
Clifford: Stabilizer = ['-ZYYYIIYX', '-IZIYXXZX', '+ZYXYIYZI', '-IIXIIYIZ', '-ZIIYXYZZ', '-IXXIYYYI', '-IYYYXYXY', '+YXXYXZYY'], Destabilizer = ['+YZIXZYYI', '-XZYIYYYX', '-YZIYXYYI', '-XIZYZYII', '+IXZXXIYY', '-XIYYXIZX', '-XZZZIZXZ', '-XXIYZYZZ']


In [48]:
with open('cliff_list_experiment.pkl', 'rb') as f:
    clifford_list_2 = pickle.load(f)

In [49]:
print(clifford_list)
print(clifford_list_2)

[{'stabilizer': ['-IXIZZYZI', '+YYYIYXYZ', '-XXZYXXYZ', '-ZYIZIXIX', '+YIZZXIXY', '+ZXZIIZXX', '-YYYZIZZI', '-YZXXIZYX'], 'destabilizer': ['+XXYIYXYI', '+YYXXIIZX', '-ZZZZYYII', '-IZZXZYII', '-XZZYZXXY', '-IZXXYIYI', '-IYYYIXXY', '+XIZZXYXX']}, {'stabilizer': ['-IYXIIIZX', '-XZYXZYZI', '+ZXZIZXZX', '+YYXYZYII', '+IXXIXYYI', '+ZIIXXZXY', '+ZYIIYIYY', '-YYXIZYYY'], 'destabilizer': ['-IIXIYZYI', '-ZIIIIXXZ', '-XYYXIZZY', '+ZYIXYIYY', '-XIYIXXZZ', '-IXZIZIII', '+IYIYXXII', '-IYYXIYYI']}, {'stabilizer': ['-YYXIYYZX', '-XIZIYYYY', '-YZYYIIXZ', '+ZXIXZZYZ', '+XYZZZYIZ', '-XZXXIIXY', '+ZZZIXXXI', '-IYZZIIYI'], 'destabilizer': ['+XYXIZIXX', '+ZZIZZYXI', '-XZIYYYYY', '+ZIYXZXII', '+YIXZXIXI', '-ZZZIYIZY', '+ZYXIXXZZ', '-XYZXYYII']}, {'stabilizer': ['+XZXZYXIY', '+XIZXZZZY', '-IYYIZYZI', '-IZXZYXII', '-YYZXIZIZ', '-XYXZYZYY', '+YXXZZIIX', '-ZYXXIIXZ'], 'destabilizer': ['+XYXZZIYX', '-IIYYXYYY', '+IZYXXZII', '-IYYXYIXX', '-XZIIZYXI', '+YXZYYZIZ', '-YIZIIYYX', '-XIZIYZXY']}, {'stabilizer': ['-XIYXZ

In [13]:
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit_ibm_runtime.fake_provider import FakeBrisbane

# Construct circuit
print(nelec)
# qubits = 2*norb
qubits = qiskit.QuantumRegister(2 * norb, name="q")
circuit = qiskit.QuantumCircuit(qubits)
circuit.append(ffsim.qiskit.PrepareHartreeFockJW(norb, nelec),qubits)
circuit.append(ffsim.qiskit.UCJOpSpinBalancedJW(operator),qubits)

# get backend
# provider = IBMProvider()
backend = FakeBrisbane()

qc_compiled = transpile(circuit, backend,optimization_level=3)

print("depth:",qc_compiled.depth(),"\ntotal gates: ",qc_compiled.size(),"\ngate breakdown: ",qc_compiled.count_ops())

(2, 2)
depth: 379 
total gates:  1067 
gate breakdown:  OrderedDict([('rz', 518), ('sx', 330), ('ecr', 182), ('x', 37)])


In [85]:
estimator = AerEstimator(approximation=True)
nelec = mol.nelec
num_orbitals = nsite
'''
Constructing the ansatz 

'''

# Make a qiskit HF reference state

initial_state = HartreeFock(
num_spatial_orbitals= num_orbitals,
num_particles=nelec , #nsite(1,1)
qubit_mapper=mapper,
)




# UCC ansatz
ansatz = UCC(
    excitations="sd",
    num_particles= nelec , # nelec
    num_spatial_orbitals=num_orbitals,
    initial_state=initial_state,
    qubit_mapper= mapper,
    reps=1,
)
cobyla = COBYLA(maxiter=1000)
        
#print("pre vqe")
    


In [86]:
vqe = VQE(estimator,ansatz, optimizer=cobyla, initial_point = np.zeros(ansatz.num_parameters))
result = vqe.compute_minimum_eigenvalue(operator= qubit_ham)
vqe_energy = result.optimal_value.real
optimal_point=result.optimal_point
print(vqe_energy+mol.energy_nuc(),optimal_point)

-2.1663079694994414 [ 6.35966677e-03 -8.69687789e-06  1.66438100e-04  5.03842385e-03  6.40899031e-03  3.91919073e-05 -1.87668478e-04
  5.04621248e-03  3.11451039e-02 -6.54614866e-02 -1.03999292e-05  3.06645802e-05  8.00135759e-02  1.86986483e-05
 -5.40940749e-02  4.84147494e-02  1.21668828e-05  4.32845634e-05  4.83834110e-02 -1.91300697e-01 -2.76545002e-05
  8.07138238e-02 -2.94174830e-05 -4.80261818e-05 -4.24091298e-02  3.14908829e-02]


In [20]:
vqe = VQE(estimator,ansatz, optimizer=cobyla, initial_point = np.zeros(ansatz.num_parameters))
result = vqe.compute_minimum_eigenvalue(operator= qubit_ham)
vqe_energy = result.optimal_value.real
optimal_point=result.optimal_point
print(vqe_energy+mol.energy_nuc(),optimal_point)

-2.1663079694994414 [ 6.35966677e-03 -8.69687789e-06  1.66438100e-04  5.03842385e-03  6.40899031e-03  3.91919073e-05 -1.87668478e-04
  5.04621248e-03  3.11451039e-02 -6.54614866e-02 -1.03999292e-05  3.06645802e-05  8.00135759e-02  1.86986483e-05
 -5.40940749e-02  4.84147494e-02  1.21668828e-05  4.32845634e-05  4.83834110e-02 -1.91300697e-01 -2.76545002e-05
  8.07138238e-02 -2.94174830e-05 -4.80261818e-05 -4.24091298e-02  3.14908829e-02]


In [66]:
repeat_attempts=4
print(vqe_energy)
for initial_param_count in range(repeat_attempts):
    #print("param count: ", initial_param_count)
    vqe = VQE(estimator,ansatz, optimizer=cobyla, initial_point = np.random.normal(0.0,np.pi/4.,ansatz.num_parameters))
    result = vqe.compute_minimum_eigenvalue(operator= qubit_ham)
    print(result.optimal_value.real)
    if result.optimal_value.real < vqe_energy:
        print("Found better than previous initial params", vqe_energy, result.optimal_value.real)
        vqe_energy = result.optimal_value.real 
        optimal_point=result.optimal_point

-4.4594092168194415
-4.396440768288837
-4.398716911372031
-4.398738865211214
-4.04893262262663


In [93]:
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit_ibm_runtime.fake_provider import FakeBrisbane

# get backend
# provider = IBMProvider()
backend = FakeBrisbane()

GS_circ = ansatz.assign_parameters(optimal_point)
GS_circ.measure_all()


# backend = AerSimulator()

# First we have to transpile the quantum circuit 
# to the low-level QASM instructions used by the 
# backend

qc_compiled = transpile(GS_circ, backend,optimization_level=3)

print("depth:",qc_compiled.depth(),"\ntotal gates: ",qc_compiled.size(),"\ngate breakdown: ",qc_compiled.count_ops())
# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = backend.run(qc_compiled, shots=1024)

# Grab the results from the job.
result_sim = job_sim.result()
counts = result_sim.get_counts(qc_compiled)
print(counts)

depth: 3127 
total gates:  4987 
gate breakdown:  OrderedDict([('rz', 2557), ('sx', 1622), ('ecr', 708), ('x', 92), ('measure', 8), ('barrier', 1)])
{'00010001': 1, '01011100': 1, '10100010': 1, '01010110': 2, '11001110': 1, '10111010': 2, '10000100': 1, '00110110': 1, '00101110': 2, '00111010': 2, '10101100': 2, '11011110': 3, '00000110': 1, '00100100': 1, '10000000': 1, '00001110': 1, '01100110': 2, '10101001': 4, '00000100': 1, '11000100': 2, '11011101': 6, '01111010': 1, '10111100': 3, '10011010': 5, '00101001': 6, '11010101': 8, '00011111': 4, '11011001': 4, '11101110': 2, '10001110': 1, '10110110': 1, '01001010': 1, '11111111': 6, '00010101': 4, '11000111': 9, '01111100': 2, '01000101': 3, '11100111': 6, '00000000': 2, '11101010': 1, '00011000': 1, '01001100': 1, '00011100': 2, '11110101': 7, '11011100': 2, '00000010': 1, '01110100': 1, '10101010': 1, '11000101': 9, '10110001': 5, '01101001': 5, '00111111': 10, '01101111': 11, '10010100': 3, '11100100': 4, '01011000': 1, '1011100

In [78]:
from qiskit_aer import AerSimulator
from qiskit import transpile

backend = AerSimulator()

# First we have to transpile the quantum circuit 
# to the low-level QASM instructions used by the 
# backend

qc_compiled = transpile(GS_circ, backend,optimization_level=3)

print("depth:",qc_compiled.depth(),qc_compiled.size(),qc_compiled.count_ops())
# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = backend.run(qc_compiled, shots=1024)

# Grab the results from the job.
result_sim = job_sim.result()
counts = result_sim.get_counts(qc_compiled)
print(counts)

depth: 949 1159 OrderedDict([('cx', 728), ('u2', 170), ('unitary', 121), ('rz', 105), ('h', 21), ('measure', 8), ('u3', 4), ('x', 2), ('barrier', 1)])
{'01011010': 4, '11000011': 3, '10101010': 2, '10010110': 3, '10011001': 1, '01100110': 6, '01101001': 10, '01010101': 45, '00110011': 950}


In [108]:
rand_clifford = qiskit.quantum_info.random_clifford(GS_circ.num_qubits)
rand_cliff_circ = rand_clifford.to_circuit()
rand_cliff_circ = transpile(rand_cliff_circ, backend,optimization_level=3)
print("\n\ndepth: ",rand_cliff_circ.depth(),"\ntotal gates: ",rand_cliff_circ.size(),"\ngate breakdown: ",rand_cliff_circ.count_ops())

rand_clifford = qiskit.quantum_info.random_clifford(GS_circ.num_qubits)
rand_cliff_circ = rand_clifford.to_circuit()
rand_cliff_circ = transpile(rand_cliff_circ, backend,optimization_level=3)
print("\n\ndepth: ",rand_cliff_circ.depth(),"\ntotal gates: ",rand_cliff_circ.size(),"\ngate breakdown: ",rand_cliff_circ.count_ops())

rand_clifford = qiskit.quantum_info.random_clifford(GS_circ.num_qubits)
rand_cliff_circ = rand_clifford.to_circuit()
rand_cliff_circ = transpile(rand_cliff_circ, backend,optimization_level=3)
print("\n\ndepth: ",rand_cliff_circ.depth(),"\ntotal gates: ",rand_cliff_circ.size(),"\ngate breakdown: ",rand_cliff_circ.count_ops())

rand_clifford = qiskit.quantum_info.random_clifford(GS_circ.num_qubits)
rand_cliff_circ = rand_clifford.to_circuit()
rand_cliff_circ = transpile(rand_cliff_circ, backend,optimization_level=3)
print("\n\ndepth: ",rand_cliff_circ.depth(),"\ntotal gates: ",rand_cliff_circ.size(),"\ngate breakdown: ",rand_cliff_circ.count_ops())

rand_clifford = qiskit.quantum_info.random_clifford(GS_circ.num_qubits)
rand_cliff_circ = rand_clifford.to_circuit()
rand_cliff_circ = transpile(rand_cliff_circ, backend,optimization_level=3)
print("\n\ndepth: ",rand_cliff_circ.depth(),"\ntotal gates: ",rand_cliff_circ.size(),"\ngate breakdown: ",rand_cliff_circ.count_ops())



depth:  231 
total gates:  450 
gate breakdown:  OrderedDict([('rz', 231), ('sx', 148), ('ecr', 65), ('x', 6)])


depth:  287 
total gates:  590 
gate breakdown:  OrderedDict([('rz', 303), ('sx', 188), ('ecr', 92), ('x', 7)])


depth:  237 
total gates:  523 
gate breakdown:  OrderedDict([('rz', 271), ('sx', 158), ('ecr', 81), ('x', 13)])


depth:  302 
total gates:  591 
gate breakdown:  OrderedDict([('rz', 313), ('sx', 182), ('ecr', 91), ('x', 5)])


depth:  166 
total gates:  376 
gate breakdown:  OrderedDict([('rz', 194), ('sx', 115), ('ecr', 59), ('x', 8)])


In [79]:
print(qc_compiled)

global phase: 3.927
        ┌─────────────┐     ┌───┐┌───────────────┐┌───┐┌──────────────┐┌───┐»
   q_0: ┤ U2(-π,-π/2) ├─────┤ X ├┤ Rz(0.0063597) ├┤ X ├┤ U2(-π/2,π/2) ├┤ X ├»
        └────┬───┬────┘┌───┐└─┬─┘└───────────────┘└─┬─┘└─┬──────────┬─┘└─┬─┘»
   q_1: ─────┤ X ├─────┤ X ├──■─────────────────────■────┤0         ├────■──»
             ├───┤     └─┬─┘                             │  Unitary │       »
   q_2: ─────┤ H ├───────■───────────────────────────────┤1         ├───────»
             ├───┤                                       └──────────┘       »
   q_3: ─────┤ H ├──────────────────────────────────────────────────────────»
        ┌────┴───┴────┐     ┌───┐ ┌──────────────┐┌───┐┌──────────────┐┌───┐»
   q_4: ┤ U2(-π,-π/2) ├─────┤ X ├─┤ Rz(0.006409) ├┤ X ├┤ U2(-π/2,π/2) ├┤ X ├»
        └────┬───┬────┘┌───┐└─┬─┘ └──────────────┘└─┬─┘└─┬──────────┬─┘└─┬─┘»
   q_5: ─────┤ X ├─────┤ X ├──■─────────────────────■────┤0         ├────■──»
             ├───┤     └─┬─┘                